In [1]:
!pip install setuptools==45.3.0

     |████████████████████████████████| 593kB 2.7MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: setuptools 50.3.0
    Uninstalling setuptools-50.3.0:
      Successfully uninstalled setuptools-50.3.0


In [2]:
!apt-get install openslide-tools
!pip install openslide-python

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libopenslide0
Suggested packages:
  libtiff-tools
The following NEW packages will be installed:
  libopenslide0 openslide-tools
0 upgraded, 2 newly installed, 0 to remove and 6 not upgraded.
Need to get 92.5 kB of archives.
After this operation, 268 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libopenslide0 amd64 3.4.1+dfsg-2 [79.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 openslide-tools amd64 3.4.1+dfsg-2 [12.7 kB]
Fetched 92.5 kB in 1s (63.9 kB/s)
Selecting previously unselected package libopenslide0.
(Reading database ... 144617 files and directories currently installed.)
Preparing to unpack .../libopenslide0_3.4.1+dfsg-2_amd64.deb ...
Unpacking libopenslide0 (3.4.1+dfsg-2) ...
Selecting previously unselected package openslide-tools.
Preparing to unpack 

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from openslide import open_slide
from openslide import __library_version__ as openslide_version
import os
from PIL import Image
from skimage.color import rgb2gray

In [4]:
#Define Image size and level to be used
IMG_SIZE = 299
level = 5

## Create Dataset using Gigapixel images from Google Drive

In [5]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [6]:
!cp -r /gdrive/My\ Drive/slides /content
slides_directory = '/content/slides'

In [7]:
import os
files = os.listdir(slides_directory)
print(len(files))
print(files)

67
['tumor_091_mask.tif', 'tumor_091.tif', 'tumor_001_mask.tif', 'tumor_001.tif', 'tumor_002_mask.tif', 'tumor_002.tif', 'tumor_005_mask.tif', 'tumor_005.tif', 'tumor_012_mask.tif', 'tumor_012.tif', 'tumor_016_mask.tif', 'tumor_016.tif', 'tumor_019_mask.tif', 'tumor_059.xml', 'tumor_002.xml', 'tumor_075.xml', 'tumor_023.xml', 'tumor_057.xml', 'tumor_005.xml', 'tumor_035.xml', 'tumor_019.xml', 'tumor_012.xml', 'tumor_016.xml', 'tumor_001.xml', 'tumor_081.xml', 'tumor_084.xml', 'tumor_019.tif', 'tumor_091.xml', 'tumor_094.xml', 'tumor_110.xml', 'tumor_096.xml', 'tumor_101.xml', 'tumor_099.xml', 'tumor_023_mask.tif', 'tumor_023.tif', 'tumor_031_mask.tif', 'tumor_031.tif', 'tumor_035_mask.tif', 'tumor_035.tif', 'tumor_038.tif', 'tumor_038 (1).xml', 'tumor_057_mask.tif', 'tumor_057.tif', 'tumor_059_mask.tif', 'tumor_059.tif', 'tumor_064_mask.tif', 'tumor_064.tif', 'tumor_075_mask.tif', 'tumor_075.tif', 'tumor_078_mask.tif', 'tumor_078.tif', 'tumor_081_mask.tif', 'tumor_081.tif', 'tumor_084_

In [8]:
tumor_mask_pairs = []

for file in files:
  if 'mask' in file:
    mask = file
    slide = file.split('.')[0][0:-5] + '.tif'
    tumor_mask_pairs.append([mask, slide])

print(len(tumor_mask_pairs))
print(tumor_mask_pairs)

21
[['tumor_091_mask.tif', 'tumor_091.tif'], ['tumor_001_mask.tif', 'tumor_001.tif'], ['tumor_002_mask.tif', 'tumor_002.tif'], ['tumor_005_mask.tif', 'tumor_005.tif'], ['tumor_012_mask.tif', 'tumor_012.tif'], ['tumor_016_mask.tif', 'tumor_016.tif'], ['tumor_019_mask.tif', 'tumor_019.tif'], ['tumor_023_mask.tif', 'tumor_023.tif'], ['tumor_031_mask.tif', 'tumor_031.tif'], ['tumor_035_mask.tif', 'tumor_035.tif'], ['tumor_057_mask.tif', 'tumor_057.tif'], ['tumor_059_mask.tif', 'tumor_059.tif'], ['tumor_064_mask.tif', 'tumor_064.tif'], ['tumor_075_mask.tif', 'tumor_075.tif'], ['tumor_078_mask.tif', 'tumor_078.tif'], ['tumor_081_mask.tif', 'tumor_081.tif'], ['tumor_084_mask.tif', 'tumor_084.tif'], ['tumor_094_mask.tif', 'tumor_094.tif'], ['tumor_096_mask.tif', 'tumor_096.tif'], ['tumor_110_mask.tif', 'tumor_110.tif'], ['tumor_101_mask.tif', 'tumor_101.tif']]


### Train Test separation

In [9]:
train_size = int(len(tumor_mask_pairs)*0.8)
train_slides = tumor_mask_pairs[:train_size]
test_slides = tumor_mask_pairs[train_size:]

In [10]:
print(len(train_slides))
print(len(test_slides))

16
5


# Generate Dataset and save to disk

In [11]:
def generate_dataset(tumor_mask_pairs, slides_directory):
  for pair in tumor_mask_pairs:
    print(pair)
    mask_path = slides_directory + '/' + pair[0]
    slide_path = slides_directory + '/' + pair[1]

    slide = open_slide(slide_path)
    tumor_mask = open_slide(mask_path)

    pos_images, mask_subimages, neg_images = extract_images(slide, tumor_mask, level)

    %cd positive/
    save_pos_images_with_flippping(pair[1], pos_images)
    
    %cd ../negative/
    save_neg_images(pair[1], neg_images)

    %cd ..

In [12]:
def extract_images(image_slide, tumor_slide, level):
  factor = 2**level
  threshold_pixels_cancerous = 0.001*(IMG_SIZE * IMG_SIZE)

  pos_images = []
  mask_subimages = []
  neg_images = []

  for i in range(0, image_slide.level_dimensions[level][0], IMG_SIZE):
    for j in range(0, image_slide.level_dimensions[level][1], IMG_SIZE):
      sub_img = read_slide(image_slide, x=i*factor, y=j*factor, level=level, width=IMG_SIZE, height=IMG_SIZE)
      tissue_pixels = find_tissue_pixels(sub_img)
      percent_tissue = len(tissue_pixels) / float(sub_img.shape[0]*sub_img.shape[1]) * 100
      # if percent_tissue > 0.1:
      if percent_tissue > 10.0:
        mask_subimg = read_slide(tumor_slide, x=i*factor, y=j*factor, level=level, width=IMG_SIZE, height=IMG_SIZE)
        mask_subimg = mask_subimg[:,:,0]
        n_tumour_pixels = np.count_nonzero(mask_subimg == 1)
        if n_tumour_pixels>threshold_pixels_cancerous:
          pos_images.append(sub_img)
          mask_subimages.append(mask_subimg)
        else:
          neg_images.append(sub_img)
  
  return pos_images, mask_subimages, neg_images

In [13]:
def read_slide(slide, x, y, level, width, height, as_float=False):
    im = slide.read_region((x,y), level, (width, height))
    im = im.convert('RGB') # drop the alpha channel
    if as_float:
        im = np.asarray(im, dtype=np.float32)
    else:
        im = np.asarray(im)
    assert im.shape == (height, width, 3)
    return im

def find_tissue_pixels(image, intensity=0.8):
    im_gray = rgb2gray(image)
    assert im_gray.shape == (image.shape[0], image.shape[1])
    indices = np.where(im_gray <= intensity)
    return list(zip(indices[0], indices[1]))

In [14]:
#save images along with some rotated variants
def save_pos_images_with_flippping(fname, pos_images):
  i=0
  for j in range(len(pos_images)):
    result = Image.fromarray((pos_images[j]).astype(np.uint8))
    result.save(fname + '_img_' + str(i) + '.bmp')
    i += 1
  
  #add images flipped left to right
  for j in range(len(pos_images)):
    flipped_image = np.fliplr(pos_images[j])
    result = Image.fromarray(flipped_image.astype(np.uint8))
    result.save(fname + '_img_' + str(i) + '.bmp')
    i += 1

  #add images flipped up down
  for j in range(len(pos_images)):
    flipped_image = np.flipud(pos_images[j])
    result = Image.fromarray(flipped_image.astype(np.uint8))
    result.save(fname + '_img_' + str(i) + '.bmp')
    i += 1
  
  #add images flipped lr and ud
  for j in range(len(pos_images)):
    flipped_image = np.fliplr(pos_images[j])
    flipped_image_2 = np.flipud(flipped_image)
    result = Image.fromarray(flipped_image_2.astype(np.uint8))
    result.save(fname + '_img_' + str(i) + '.bmp')
    i += 1
  return i

In [15]:
def save_neg_images(fname, neg_images):
  i=0
  for j in range(len(neg_images)):
    result = Image.fromarray((neg_images[j]).astype(np.uint8))
    result.save(fname + '_img_' + str(i) + '.bmp')
    i += 1

In [16]:
!mkdir 'camelyon_small_data'
%cd 'camelyon_small_data'
!mkdir 'train'
!mkdir 'test'

/content/camelyon_small_data


In [17]:
%cd 'train'
!mkdir 'positive'
!mkdir 'negative'
generate_dataset(train_slides, slides_directory)

/content/camelyon_small_data/train
['tumor_091_mask.tif', 'tumor_091.tif']
/content/camelyon_small_data/train/positive
/content/camelyon_small_data/train/negative
/content/camelyon_small_data/train
['tumor_001_mask.tif', 'tumor_001.tif']
/content/camelyon_small_data/train/positive
/content/camelyon_small_data/train/negative
/content/camelyon_small_data/train
['tumor_002_mask.tif', 'tumor_002.tif']
/content/camelyon_small_data/train/positive
/content/camelyon_small_data/train/negative
/content/camelyon_small_data/train
['tumor_005_mask.tif', 'tumor_005.tif']
/content/camelyon_small_data/train/positive
/content/camelyon_small_data/train/negative
/content/camelyon_small_data/train
['tumor_012_mask.tif', 'tumor_012.tif']
/content/camelyon_small_data/train/positive
/content/camelyon_small_data/train/negative
/content/camelyon_small_data/train
['tumor_016_mask.tif', 'tumor_016.tif']
/content/camelyon_small_data/train/positive
/content/camelyon_small_data/train/negative
/content/camelyon_smal

In [18]:
%cd ../test
!mkdir positive
!mkdir negative
generate_dataset(test_slides, slides_directory)

/content/camelyon_small_data/test
['tumor_084_mask.tif', 'tumor_084.tif']
/content/camelyon_small_data/test/positive
/content/camelyon_small_data/test/negative
/content/camelyon_small_data/test
['tumor_094_mask.tif', 'tumor_094.tif']
/content/camelyon_small_data/test/positive
/content/camelyon_small_data/test/negative
/content/camelyon_small_data/test
['tumor_096_mask.tif', 'tumor_096.tif']
/content/camelyon_small_data/test/positive
/content/camelyon_small_data/test/negative
/content/camelyon_small_data/test
['tumor_110_mask.tif', 'tumor_110.tif']
/content/camelyon_small_data/test/positive
/content/camelyon_small_data/test/negative
/content/camelyon_small_data/test
['tumor_101_mask.tif', 'tumor_101.tif']
/content/camelyon_small_data/test/positive
/content/camelyon_small_data/test/negative
/content/camelyon_small_data/test


## Train Model

In [20]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
print(tf.__version__)

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import datasets, layers, models

2.3.0


In [28]:
data_root_train = '/content/camelyon_small_data/train'
data_root_validation = '/content/camelyon_small_data/test'

train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        data_root_train,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=32,
        class_mode='binary'
        )

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = train_datagen.flow_from_directory(
        data_root_validation,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=32,
        class_mode='binary'
        )

i=0
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    #print(data_batch[0])
    print('labels batch shape:', labels_batch.shape)
    print(labels_batch)
    if i==3:
      break
    else:
      i+=1


Found 1137 images belonging to 2 classes.
Found 500 images belonging to 2 classes.
data batch shape: (32, 299, 299, 3)
labels batch shape: (32,)
[0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1.
 0. 1. 0. 0. 0. 0. 1. 1.]
data batch shape: (32, 299, 299, 3)
labels batch shape: (32,)
[0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0.
 1. 1. 0. 0. 1. 0. 0. 0.]
data batch shape: (32, 299, 299, 3)
labels batch shape: (32,)
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1. 0.
 0. 1. 0. 0. 0. 0. 0. 0.]
data batch shape: (32, 299, 299, 3)
labels batch shape: (32,)
[1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 1. 0. 1. 1.]


In [45]:
# Class weights when flipped images are added to pos images
pos = sum(train_generator.classes)
neg = len(train_generator.classes) - pos
total = pos + neg

weight_for_0 = (1 / neg)*(total)/2.0 
weight_for_1 = (1 / pos)*(total)/2.0

class_weight = {0: weight_for_0, 1: weight_for_1}
print(class_weight)

{0: 0.7168978562421185, 1: 1.6526162790697674}


In [29]:
METRICS = [
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
]

In [34]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_auc', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

In [46]:
base_model = tf.keras.applications.InceptionV3(input_shape=(IMG_SIZE, IMG_SIZE, 3),
                                               include_top=False,
                                               weights='imagenet')

base_model.trainable = False
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid')

# build a new model reusing the pretrained base
model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=METRICS)

history = model.fit(
      train_generator,
      epochs=25,
      steps_per_epoch=train_generator.n//train_generator.batch_size+1,
      validation_data=validation_generator,
      validation_steps = validation_generator.n//validation_generator.batch_size+1,
      class_weight=class_weight,
      callbacks = [early_stopping])

Epoch 1/25
36/36 [==============================] - 6s 172ms/step - loss: 0.6542 - tp: 492.0000 - fp: 387.0000 - tn: 662.0000 - fn: 96.0000 - accuracy: 0.7049 - precision: 0.5597 - recall: 0.8367 - auc: 0.8193 - val_loss: 0.6698 - val_tp: 225.0000 - val_fp: 113.0000 - val_tn: 143.0000 - val_fn: 19.0000 - val_accuracy: 0.7360 - val_precision: 0.6657 - val_recall: 0.9221 - val_auc: 0.8094
Epoch 2/25
36/36 [==============================] - 4s 123ms/step - loss: 0.5990 - tp: 286.0000 - fp: 143.0000 - tn: 650.0000 - fn: 58.0000 - accuracy: 0.8232 - precision: 0.6667 - recall: 0.8314 - auc: 0.9222 - val_loss: 0.6620 - val_tp: 201.0000 - val_fp: 95.0000 - val_tn: 161.0000 - val_fn: 43.0000 - val_accuracy: 0.7240 - val_precision: 0.6791 - val_recall: 0.8238 - val_auc: 0.8042
Epoch 3/25
36/36 [==============================] - 4s 123ms/step - loss: 0.5872 - tp: 291.0000 - fp: 116.0000 - tn: 677.0000 - fn: 53.0000 - accuracy: 0.8514 - precision: 0.7150 - recall: 0.8459 - auc: 0.9361 - val_loss: